In [1]:
from fastai2.text.all import *

In [4]:
path =  Path('/home/jupyter/VLP')
df  = pd.read_csv(path/'captioned.csv')
df

,id,img,label,text,caption,is_valid,tex_cap
0,42953,img/42953.png,0.0,its their character not their color that matters,A man in a black shirt is looking at the camera .,False,A man in a black shirt is looking at the camera . <meme> its their character not their color that matters
1,23058,img/23058.png,0.0,don't be afraid to love again everyone is not like your ex,A man in a suit is talking to a woman in a white shirt .,False,A man in a suit is talking to a woman in a white shirt . <meme> don't be afraid to love again everyone is not like your ex
2,13894,img/13894.png,0.0,putting bows on your pet,A cat with a red bow tied around its neck sits next to a pillow .,False,A cat with a red bow tied around its neck sits next to a pillow . <meme> putting bows on your pet
3,37408,img/37408.png,0.0,i love everything and everybody! except for squirrels i hate squirrels,A black dog is looking at the camera .,False,A black dog is looking at the camera . <meme> i love everything and everybody! except for squirrels i hate squirrels
4,82403,img/82403.png,0.0,"everybody loves chocolate chip cookies, even hitler",A man in a suit is talking .,False,"A man in a suit is talking . <meme> everybody loves chocolate chip cookies, even hitler"
...,...,...,...,...,...,...,...
9995,3869,img/03869.png,NaN,a mother's love for the child is a divine thing,Three squirrels are sitting on a table .,NaN,Three squirrels are sitting on a table . <meme> a mother's love for the child is a divine thing
9996,23817,img/23817.png,NaN,sea monkeys,A dog shaking off water .,NaN,A dog shaking off water . <meme> sea monkeys
9997,56280,img/56280.png,NaN,little miss muffet sat on her tuffet,A man in a blue suit is sitting in a chair .,NaN,A man in a blue suit is sitting in a chair . <meme> little miss muffet sat on her tuffet
9998,29384,img/29384.png,NaN,they're in a row,Two green and yellow portable toilets .,NaN,Two green and yellow portable toilets . <meme> they're in a row


In [27]:
df_train_val, test_df = df[:9000], df[9000:]

## Language Model

In [20]:
bad_file = Path('bad_words.txt').read_text() #https://www.freewebheaders.com/bad-words-list-and-page-moderation-words-list-for-facebook/
bad_words = set(bad_file.split(','))
len(bad_words)

1555

In [18]:
class ExplToken(Transform):
    def __init__(self, bad_words):
        self.bad_words = bad_words
    def encodes(self, tokens):
        res =[]
        for t in tokens:
            if t in bad_words: 
                res.append('xxexpl')
                #print('adding for', t)
            res.append(t)
        return res

In [21]:
blocks = [TextBlock.from_df('text', None, is_lm=True, seq_len=80)]
tfms = blocks[0].type_tfms
tfms[0].special_toks.append('xxexpl')
blocks[0].type_tfms = tfms[:1] + [ExplToken(bad_words)] + tfms[1:]
splitter = RandomSplitter(.1)
dblock = DataBlock(blocks=blocks, get_x=ColReader('text'), splitter=splitter)
dls_lm = DataLoaders.from_dblock(dblock, df, seq_len=80, bs=128)

In [22]:
dls_lm.show_batch(max_n=10)

,text,text_
0,xxbos h xxrep 3 e a xxrep 4 d shoulders wheel and frame wheel and frame xxbos xxunk xxunk a xxexpl slave master xxunk a xxexpl slave whose hands and legs are xxunk xxbos when you see another black person getting their xxexpl ass whipped by the cops stop breaking the law xxbos go home xxexpl stoned eyes give nothing away xxbos there is xxunk xxunk among islamic xxunk about xxunk homosexuals should be burned or thrown off buildings xxbos,h xxrep 3 e a xxrep 4 d shoulders wheel and frame wheel and frame xxbos xxunk xxunk a xxexpl slave master xxunk a xxexpl slave whose hands and legs are xxunk xxbos when you see another black person getting their xxexpl ass whipped by the cops stop breaking the law xxbos go home xxexpl stoned eyes give nothing away xxbos there is xxunk xxunk among islamic xxunk about xxunk homosexuals should be burned or thrown off buildings xxbos brother
1,to feminism where everything is sexist and nobody understands but them xxbos 49 people were killed at christchurch muslim muslim 49 people were killed at christchurch mosques xxbos i tried to play uno with my mexican friends but they kept stealing all of the green cards xxbos name something females say before xxexpl sucking xxexpl dick xxbos my favorite vegetable is me ! xxbos xxunk farm equipment xxbos i told my xxexpl gay friend that i can turn fruits in,feminism where everything is sexist and nobody understands but them xxbos 49 people were killed at christchurch muslim muslim 49 people were killed at christchurch mosques xxbos i tried to play uno with my mexican friends but they kept stealing all of the green cards xxbos name something females say before xxexpl sucking xxexpl dick xxbos my favorite vegetable is me ! xxbos xxunk farm equipment xxbos i told my xxexpl gay friend that i can turn fruits in to
2,"brains xxbos happy national women 's day just remember , if it has an apple , it has a xxunk xxbos new seatbelt design : 45 % less car accidents xxbos from here i will see muslims xxexpl fuck up europe , attack innocent people & europeans waking up xxbos sharing and xxunk xxbos left the kitchen too many times xxbos master chef with low rating master chef with high rating xxbos indian xxunk finally finding the right xxunk you","xxbos happy national women 's day just remember , if it has an apple , it has a xxunk xxbos new seatbelt design : 45 % less car accidents xxbos from here i will see muslims xxexpl fuck up europe , attack innocent people & europeans waking up xxbos sharing and xxunk xxbos left the kitchen too many times xxbos master chef with low rating master chef with high rating xxbos indian xxunk finally finding the right xxunk you need"
3,"they are the biggest terrorists in the history of the world . we have entire history of books showing this xxunk xxbos let 's play the game of flags xxbos say allahu akbar again … … go ahead … say it ! xxbos "" xxexpl fuck it "" - my final thought before making most xxunk xxbos xxexpl rape helping them have xxexpl sex for generations xxbos xxunk xxunk are xxunk by drug xxunk and gang members because they can","are the biggest terrorists in the history of the world . we have entire history of books showing this xxunk xxbos let 's play the game of flags xxbos say allahu akbar again … … go ahead … say it ! xxbos "" xxexpl fuck it "" - my final thought before making most xxunk xxbos xxexpl rape helping them have xxexpl sex for generations xxbos xxunk xxunk are xxunk by drug xxunk and gang members because they can easily"
4,"america xxbos playing ' guess who ' in 2018 is your character keep guessing you a man or transphobic a woman ? bigot … there are xxunk genders xxbos xxunk days under investigation , constant media slander , no collusion ! this man deserves an xxunk ! xxbos i 'm dreaming of a white christmas xxbos her : he 's probably thinking about other girls … him : how do blind people know when to stop wiping their xxexpl ass","xxbos playing ' guess who ' in 2018 is your character keep guessi

In [23]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3, 
    metrics=[accuracy, Perplexity()])
learn.fine_tune(10, 1e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,6.437484,5.531651,0.137569,252.560669,00:09


epoch,train_loss,valid_loss,accuracy,perplexity,time
0,5.330438,5.057281,0.173373,157.162689,00:10
1,5.111370,4.754369,0.199442,116.090355,00:09
2,4.904170,4.477214,0.216934,87.989220,00:09
3,4.701416,4.310301,0.241138,74.462883,00:10
4,4.524222,4.210215,0.250419,67.370995,00:09
5,4.370862,4.143959,0.258483,63.051952,00:09
6,4.237361,4.100010,0.264978,60.340916,00:09
7,4.123493,4.078314,0.268926,59.045826,00:09
8,4.031217,4.068979,0.270898,58.497223,00:10
9,3.960330,4.067474,0.270598,58.409229,00:09


In [24]:
learn.fit_one_cycle(5, 1e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.748329,4.040541,0.275107,56.857079,00:09
1,3.704435,3.990178,0.284778,54.064529,00:09
2,3.648173,3.950942,0.291469,51.984314,00:09
3,3.584869,3.933376,0.294182,51.079121,00:09
4,3.537372,3.929737,0.295516,50.893608,00:09


In [25]:
learn.fit_one_cycle(5, 1e-4)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.421221,3.928167,0.295670,50.813755,00:09
1,3.418870,3.924462,0.296199,50.625813,00:09
2,3.413141,3.921483,0.296366,50.475224,00:09
3,3.406165,3.919898,0.297310,50.395283,00:09
4,3.403815,3.919589,0.297127,50.379711,00:09


In [26]:
learn.save_encoder('text_emb_lm')

## Classifier

In [28]:
blocks = [TextBlock.from_df('text', vocab = dls_lm.vocab, is_lm=False, seq_len=80), CategoryBlock]
tfms = blocks[0].type_tfms
tfms[0].special_toks.append('xxexpl')
blocks[0].type_tfms = tfms[:1] + [ExplToken(bad_words)] + tfms[1:]
splitter = ColSplitter('is_valid')
dblock = DataBlock(blocks=blocks, get_x=ColReader('text'), get_y=ColReader('label'), splitter=splitter)
dls = DataLoaders.from_dblock(dblock, df_train_val, path='.', seq_len=80, bs=128)

In [29]:
dls.show_batch()

,text,category
0,"xxbos housing , free gas , free xxunk , free healthcare and free education for my wives and children . to show xxunk for your xxunk , i 'll xxunk your 12 year old daughters , blow up your xxunk , planes and buses and xxunk hate through a xxunk xxunk in your local city and town xxunk with the purpose of turning your xxunk country into the same xxexpl shithole i xxunk took xxunk from , allahu akbar !",1.0
1,"xxbos "" nobody is trying to take your guns ! "" "" we only want to make you xxunk them , xxunk xxunk , ban xxunk guns , xxunk magazine xxunk , xxunk xxunk them , ban or xxunk ammo , make other xxunk laws , and , if we catch you xxunk any of these made - up rules , throw you in prison … . at which point we will take your guns ! "" xxpad xxpad xxpad",0.0
2,"xxbos i guess it 's all ok to demand that president trump release his taxes i xxunk $ 16 billion in 2018 to fix my rat xxunk xxexpl shithole district in west baltimore and i never did it .. asking me where the money xxunk to , makes you all a bunch of racists xxrep 4 ! but i guess it 's not ok to ask this guy what he did with baltimore 's money xxpad xxpad xxpad xxpad xxpad",0.0
3,xxbos president trump accused of xxexpl sexual assault right before the election xxunk xxunk accused xxexpl sexual harassment right before the election right before getting voted in by the senate xxunk xxunk accused of xxexpl sexual assault right before senate election in alabama brett kavanaugh accused of xxexpl sexual assault right before getting voted in by the senate still convinced that the xxunk are telling the truth ? xxunk e xxunk monday at xxunk pm xxpad xxpad xxpad xxpad xxpad,0.0
4,"xxbos if a phobia is a fear , how can i be "" xxunk "" or transphobic "" ? who xxunk a man in a dress ? ! why would i fear a man who takes it up the xxexpl ass by another man ? xxunk , yes definitely . fear ? no way . the only fear i have is the xxunk of what society will look like if these people keep xxunk xxpad xxpad xxpad xxpad xxpad xxpad",1.0
5,xxbos xxunk you killed me so the jew could control your media so the jews could control your banks so the jews control you money so the jews could control your government so your children could die for israel so foreigners could destroy your cities so foreigners could run your civil and public services so you become a prisoner in your own homes so your people and race are lost forever xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad,1.0
6,"xxbos i had twins with my third husband , which i xxunk while married to my first husband , and xxunk while married to my second husband , who adopted them before i divorced him and married my third husband . but that was then , now i 'm married to my xxunk husband . i 'm just an xxunk who believes in the xxunk of marriage xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad",1.0
7,"xxbos if i can sexually harass this many women and xxexpl molest this many children as vice president , just imagine how many i could do as president ! but what am i telling you for ? you 're the first xxexpl gay president , married to the first ' first xxexpl tranny , ' and most people do n't even know it ! xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad",1.0
8,"xxbos why do people think it 's okay to call babies he or she ? they ca n't speak yet so they ca n't say their xxunk gender please xxunk to them as xxunk or xxunk until they can say their xxunk xxunk otherwise you 're xxunk and transphobic well , there it is the xxunk thing i 'll read today xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad",0.0


In [36]:
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.95, metrics=[accuracy, RocAucBinary()])
learn = learn.load_encoder('text_emb_lm')

In [37]:
learn.fine_tune(5, 1e-2)

epoch,train_loss,valid_loss,accuracy,roc_auc_score,time
0,0.914062,0.700204,0.550000,0.580736,00:03


epoch,train_loss,valid_loss,accuracy,roc_auc_score,time
0,0.778042,0.752308,0.538000,0.580688,00:04
1,0.690990,0.755896,0.540000,0.590480,00:04
2,0.639129,0.775662,0.546000,0.603536,00:04
3,0.623369,0.741682,0.550000,0.609552,00:04
4,0.613496,0.740376,0.548000,0.612976,00:04


In [61]:
model = learn.model[0].module.eval()
model

AWD_LSTM(
  (encoder): Embedding(3832, 400, padding_idx=1)
  (encoder_dp): EmbeddingDropout(
    (emb): Embedding(3832, 400, padding_idx=1)
  )
  (rnns): ModuleList(
    (0): WeightDropout(
      (module): LSTM(400, 1152, batch_first=True)
    )
    (1): WeightDropout(
      (module): LSTM(1152, 1152, batch_first=True)
    )
    (2): WeightDropout(
      (module): LSTM(1152, 400, batch_first=True)
    )
  )
  (input_dp): RNNDropout()
  (hidden_dps): ModuleList(
    (0): RNNDropout()
    (1): RNNDropout()
    (2): RNNDropout()
  )
)

In [62]:
bx, by = dls.one_batch()
bx.shape

torch.Size([128, 80])

In [63]:
bout = model(bx)

In [64]:
bout.shape

torch.Size([128, 80, 400])

In [67]:
len(model.hidden)

3

2

In [68]:
[x.shape for x  in model.hidden]

AttributeError: 'tuple' object has no attribute 'shape'